<a href="https://colab.research.google.com/github/lucgbrl/Foundation/blob/master/Trabalho_Extra_de_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

# Trabalho Extra de IC com TensorFlow - 356726

Universidade Federal do Ceará - Campus Sobral
Lucas Gabriel Guilherme dos Santos - 356726
Engenharia da Computação
Disciplina: Inteligência Computacional
Prof. Jarbas Joaci

Trabalho Extra - Entrega até 11/12/2019
Semestre: 2019.2



Este documento utiliza [Keras](https://www.tensorflow.org/guide/keras/overview) para:

1. Construir uma rede neural para classificar imagens.
2. Treinar esta rede neural.
3. Avaliar a acurácia do modelo.

1 Introdução 

Este é um caderno do [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb). Por meio dele pode-se executar programas em Python diretamente no servidor, sendo esta uma ótima forma de aprender a utilizar  TensorFlow. 

As importações serão realizadas no bloco de código abaixo. Por meio da biblioteca TensorFlow e Keras serão gerados os modelos de rede neural convolucional. Será empregada a biblioteca Matplotlib para exibição dos valores numéricos e a biblioteca numpy para operações matemáticas auxiliares.

Também é importado o sistema de arquivos do Google Drive, onde foi realizado o envio do conjunto de dados de imagens, disponibilizado na url [datasets de IC](https://www.dropbox.com/s/n1elpgp4xxagbrv/mnist.rar?dl=0).

Para acessar arquivos no Google Drive a partir do Google Colab é necessário gerar um token de identificação, normalmente após a primeira execução é exibido um link de acesso para o Drive, onde é solicitado a permissão do proprietário.

O Token gerado para a pasta MNIST foi: 4/uAHrA5VaxjyK5wl-dwQ5BjfL3qBR5inzX0l2Ux18kvmlkdVU585ae5o

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os 
import cv2
from google.colab import drive
from google.colab import files 
import matplotlib.pyplot as plt

# Lucas Gabriel - 356726
# Engenharia da Computação - UFC Sobral
# montando o sistema de arquivos no Google Drive

# A variável image_path é global, bem como a variável image_data

drive.mount('/content/drive')
image_path = 'drive/My Drive/mnist'

def loadImages(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_files = sorted([os.path.join(path, 'train', file)
                          for file in os.listdir(path + "/train")
                          if file.endswith('.png')])
    return image_files

# Em seguida verificar a melhor versão do TensorFlow para o Google Colab

try:
  # % A versão 2.0 e 2.x+ só existem no Google Colab, as mesmas não podem ser obtidas para execução local.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

As funções definidas abaixo (display e display_one) são responsáveis por exibir imagens obtidas a partir do banco de testes. O banco encontra-se numa pasta chamada MNIST no Google Drive, dentro de uma subpasta chamada test. 

A primeira função recebe como parâmetro duas imagens e dois títulos, definidos previamente. A segunda função recebe somente uma imagem como parâmetro e retorna a visualização desta e o títurlo pré definido.


In [0]:
# Exibe duas imagens e seus títulos, lado a lado
def display(a, b, title1 = "mnist_100", title2 = "mnist_5"):
    plt.subplot(121), plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(b), plt.title(title2)
    plt.xticks([]), plt.yticks([])
    plt.show()

# Exibe uma imagem e seu títurlo
def display_one(a, title1 = "Carregando uma imagem do banco do DropBox"):
    plt.imshow(a), plt.title(title1)
    plt.show()


A função abaixo realiza o pré processamento das imagens no banco fornecido no Dropbox, realizando o redimensionamento de 30 x 30 pixels para 28 x 28 pixels. O procedimento utiliza uma função da biblioteca OpenCV na versão 2.


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [0]:
#lê imagens entre 0 e o tamanho da lista passada em Data 

def preprocess(data):
  lim = len(data)
  # a linha a seguir lê as imagens no diretório de trabalho definido em image_path, localizada no Google Drive
  img = [cv2.imread(i, cv2.IMREAD_UNCHANGED) for i in data[:lim]]
  #img = [cv2.imread(data, cv2.IMREAD_UNCHANGED)]
  
  try:
    print('Imagem em dimensões originais:', img.shape)
  except AttributeError:
    print('Dimensões não processadas')
  
  # A partir daqui serão definidas as novas dimensões
  altura, largura = 28, 28
  dim = (altura, largura)
  res_image = []
  for i in range(len(img)):
    res = cv2.resize(img[i], dim, interpolation=cv2.INTER_LINEAR)
    res_image.append(res)

  #checando se o redimensionamento ocorreu com sucesso

  try:
    print('Redimensionando a imagem 1', res_image[0].shape)
  except AttributeError:
    print('Dimensões não encontradas')

  original = res_image[0]
  display_one(original)

#fim da implementação da função



In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

"""
train data -> mnist.test_data()
#test data -> dropbox.data()
#labels = y_test 
"""


In [0]:
print("#----------------------------------")
print('Imagens a partir do banco de testes do MNIST')
display(x_test[1], x_test[2])

print("#----------------------------------")
print('Imagem a partir do banco de teste do DROPBOX')
imagens = loadImages(image_path)
preprocess(imagens)


A seguir será definida uma função para realizar o tratamento das imagens de amostra, carregadas na lista retornada pela função loadImages:

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Train and evaluate the model:

In [0]:
#utilizando o conjunto de 10k

model.fit(x_test, y_test, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

"""
Utilizando o conjunto de 60k
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)
"""

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

In [0]:
#calcular perca e precisão do modelo


var_loss, var_accuracy = model.evaluate(x_test, y_test)
print(var_loss, var_accuracy)

O modelo treinado acima é então salvo para que possa ser empregado futuramente em novas implementações

In [0]:
#save a model 

model.save('356726.model')

In [0]:
novo_modelo = tf.keras.models.load_model('356726.model')
predicoes = novo_modelo.predict([x_test])

print(predicoes[100])

Utilizando numpy para 

In [0]:
# este dado vem a partir do mnist.test (base com 10k imagens de dígitos)
# como o banco de 10k foi utilizado para o treino é necessário utilizar outro banco
# para validação, no caso o dropbox 

print(np.argmax(predicoes[1]))
plt.imshow(x_test[1])
plt.show()

images = loadImages(image_path)
print(images[1])
preprocess(images[1])



Algumas funções auxiliares foram obtidas a partir do tutorial disponível em:

https://colab.research.google.com/drive/1b8pVMMoR37a3b9ICo8TMqMLVD-WvbzTk#scrollTo=3E94MjxkR_hI

Outras informações relevantes foram obtidos a partir do Stack Overflow:

https://stackoverflow.com/questions/48257255/how-to-import-pre-downloaded-mnist-dataset-from-a-specific-directory-or-folder